In [7]:
from tensorflow.keras.datasets import mnist
(train_images, Y_train), (test_images, Y_test) = mnist.load_data()
train_images = train_images.reshape((train_images.shape[0], -1))/255.
test_images = test_images.reshape((test_images.shape[0], -1))/255.

11501568/11490434 [==============================] - 1s 0us/step


In [1]:
import tensorflow as tf
dims = (28,28, 1)
n_components = 2
encoder = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=dims),
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=n_components),
])
encoder.summary()
from umap.parametric_umap import ParametricUMAP
embedder = ParametricUMAP(encoder=encoder, dims=dims, n_components=n_components, n_training_epochs=1, verbose=True)
embedding = embedder.fit_transform(train_images)
embedding = embedder.embedding_

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3211776   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 3,549,954
Trainable params: 3,549,954
Non-trainable params: 0
______________________________________________

In [2]:
embedder.save('/home/jovyan/work/dawn2023')

INFO:tensorflow:Assets written to: /home/jovyan/work/dawn2023/encoder/assets
Keras encoder model saved to /home/jovyan/work/dawn2023/encoder
INFO:tensorflow:Assets written to: /home/jovyan/work/dawn2023/parametric_model/assets
Keras full model saved to /home/jovyan/work/dawn2023/parametric_model
Thu Aug 17 17:26:54 2023 Worst tree score: 0.59861667
Thu Aug 17 17:26:54 2023 Mean tree score: 0.60965882
Thu Aug 17 17:26:54 2023 Best tree score: 0.61911667
Thu Aug 17 17:26:57 2023 Forward diversification reduced edges from 900000 to 384287
Thu Aug 17 17:26:59 2023 Reverse diversification reduced edges from 384287 to 384287
Thu Aug 17 17:27:00 2023 Degree pruning reduced edges from 421098 to 421095
Thu Aug 17 17:27:00 2023 Resorting data and graph based on tree order
Thu Aug 17 17:27:00 2023 Building and compiling search function
Thu Aug 17 17:27:05 2023 Building and compiling search function
Pickle of ParametricUMAP model saved to /home/jovyan/work/dawn2023/model.pkl


In [9]:
from umap.parametric_umap import load_ParametricUMAP
embedder = load_ParametricUMAP('/home/jovyan/work/dawn2023/models')
embedding = embedder.embedding_

Fri Aug 18 13:48:37 2023 Building and compiling search function
Pickle of ParametricUMAP model loaded from /home/jovyan/work/dawn2023/models/model.pkl
Keras encoder model loaded from /home/jovyan/work/dawn2023/models/encoder
Keras full model loaded from /home/jovyan/work/dawn2023/models/parametric_model


In [13]:
import numpy as np
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.events import Tap
from bokeh.plotting import figure
from bokeh.layouts import row
import matplotlib.pyplot as plt
output_notebook()

Loading BokehJS ...

In [14]:
cmap = plt.cm.get_cmap('rainbow', len(np.unique(Y_train.astype(int))))
rgb_colors_list = [tuple(int(255 * x) for x in cmap(i)[:3]) for i in Y_train.astype(int)]

p = figure(tools="tap", title="Select a point")
p.circle(embedding[:,0], embedding[:,1], size=8, color=rgb_colors_list)

p2 = figure(x_range=(0,1), y_range=(0,1), width=400, height=400, title="Transformation result")
image = p2.image(image=[np.zeros((28, 28))], x=[0], y=[0], dw=[1], dh=[1])

# Define the callback function
def update(event):
    # Get coordinates of mouse click
    x, y = event.x, event.y

    result_image = embedder.inverse_transform([x,y])
    image.data_source.data['image'] = [result_image.reshape(28, 28)]
    push_notebook()

# Attach the callback function to tap events on the plot
p.on_event(Tap, update)

# Combine the plots within a Bokeh layout
layout = row(p, p2)

show(layout, notebook_handle=True)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

